# RAG : Retrieval Augmented Generation

**(with Open-source LLMs)**

## Learning Objectives

At the end of the experiment, you will be able to:

1. Load the Documents
2. Splitting the documents into chunks
3. Embedding the chunks and storing them in vector db
4. Retrieving the relevant chunks to the query
 * Addressing Diversity
 * Addressing Specificity
5. Connecting with LLM to get a final grounded answer

## Introduction

> **RAG diagram:**
>
> <img src='https://drive.google.com/uc?id=1sCVvpsmtZEU1WSK1FFGMGHbEjrgtCNLi'>

> **Vector Store and Retrieval:**
>
> <img src='https://drive.google.com/uc?id=1_zX5gtSNrV8Qdx7Nz4_gMR8dCwvxCDS7' width=750px>

> **Embedding Model:**
>
> <img src='https://drive.google.com/uc?id=1HnvjGJ4HmpS-0wndpH-Q8cKMwIwWkTUe'>

> **Retrieval in Action:**
>
> <img src='https://drive.google.com/uc?id=1ry2TWFsewwqYP3Lw9muuPmbyuQqXwnYV' width=800px>

> **Example workflow with embedding model:**
>
><br>
>
> <img src='https://drive.google.com/uc?id=1zTuMMX54L2HrnmCYktTxVfMVrkIz8w15' width=600px>

### Install Dependencies

In [1]:
%%capture
!pip -q install langchain-core
!pip -q install langchain-community
!pip -q install sentence-transformers
!pip -q install langchain-huggingface
!pip -q install langchain-chroma
!pip -q install chromadb
!pip -q install pypdf

### Import Required Packages

In [2]:
import os
import numpy as np
from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

#### **Authentication for Huggingface API**

In [ ]:
# Add you HF token in Colab Secrets with name 'HF_TOKEN' then run this cell

import os
from google.colab import userdata

hfapi_key = userdata.get('HF_TOKEN')
os.environ["HF_TOKEN"] = hfapi_key
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hfapi_key
del hfapi_key

### Prepare Open Source LLM

In [ ]:
# importing HuggingFace model abstraction class from langchain
from langchain_huggingface import HuggingFaceEndpoint

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",       # Model card: https://huggingface.co/HuggingFaceH4/zephyr-7b-beta
    task="text-generation",
    max_new_tokens = 512,
    top_k = 30,
    temperature = 0.1,
    repetition_penalty = 1.03,
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# General query
response = llm.invoke("How to learn programming? give 5 points")
print(response)

.

1. Start with the basics: Before diving into complex programming concepts, it's essential to understand the fundamentals. Learn the syntax, data types, variables, and control structures of the programming language you want to learn.

2. Practice, practice, practice: The more you code, the better you become. Set aside time each day to work on coding projects, even if they're small. This will help you build confidence and develop a habit of coding regularly.

3. Collaborate with others: Join online communities, attend meetups, or find a study group to connect with other programmers. Collaborating with others can provide valuable insights, feedback, and learning opportunities.

4. Learn from examples: Study code written by experienced developers to understand how they structure their programs and solve problems. This can help you learn best practices and avoid common mistakes.

5. Stay up-to-date: Keep up with the latest trends and technologies in the programming world. Attend conferen

In [ ]:
# Specific query
llm.invoke("What is PCA?")

"\n\nPCA (Parent-Child Assistance) is a program designed to help low-income families with children ages 3 through kindergarten enroll in and graduate from high-quality early learning programs. The program provides families with the resources and support they need to ensure their children are prepared for school and have the opportunity to succeed academically and in life.\n\nHow does PCA work?\n\nPCA offers a range of services to families, including:\n\n1. Early Learning Scholarships: Families can apply for scholarships to cover the cost of high-quality early learning programs. Scholarships are awarded based on income and other factors.\n\n2. Parent Coaching: PCA provides parents with coaching and support to help them navigate the early learning system and make informed decisions about their child's education.\n\n3. Resource and Referral Services: PCA connects families with local resources and referrals to help them access additional support and services.\n\n4. Professional Development

### **Loading the documents**

[PDF Loader](https://python.langchain.com/docs/how_to/document_loader_pdf/)

In [ ]:
# UPLOAD the Docs first to this notebook, then run this cell

from langchain_community.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    PyPDFLoader("/content/pca_d1.pdf"),
    PyPDFLoader("/content/ens_d2.pdf"),
    PyPDFLoader("/content/ens_d2.pdf"),    # Loading duplicate documents on purpose
]

docs = []
for loader in loaders:
    docs.extend(loader.load())


In [ ]:
len(docs)        # 7 pages were there in total from above documents

7

In [ ]:
docs

[Document(metadata={'source': '/content/pca_d1.pdf', 'page': 0}, page_content=' \n1  \n N  \n1 Principal  Component  Analysis  \nIn real world data analysis tasks we analyze complex data i.e. multi dimensional data. We plot the  \ndata and find various patterns in it or use it to train some machine learning models.  One way to  \nthink  about  dimensions  is that  suppose  you have  an data  point  x , if we consider  this data  point  as \na physical  object  then  dimensions  are merely  a basis  of view,  like where  is the data  located  when  \nit is observed  from  horizontal  axis or vertical  axis.  \nAs the dimensions  of data  increases,  the difficulty  to visualize  it and perform  computations  on \nit also increases.  So, how  to reduce  the dimensions  of a data: - \n• Remove  the redundant  dimensions  \n• Only keep the most important dimensions  \nLet us first try to understand  some  terms: - \nVariance : It is a measure of the variability or it simply measures how sp

In [ ]:
print(docs[0].page_content)

 
1  
 N  
1 Principal  Component  Analysis  
In real world data analysis tasks we analyze complex data i.e. multi dimensional data. We plot the  
data and find various patterns in it or use it to train some machine learning models.  One way to  
think  about  dimensions  is that  suppose  you have  an data  point  x , if we consider  this data  point  as 
a physical  object  then  dimensions  are merely  a basis  of view,  like where  is the data  located  when  
it is observed  from  horizontal  axis or vertical  axis.  
As the dimensions  of data  increases,  the difficulty  to visualize  it and perform  computations  on 
it also increases.  So, how  to reduce  the dimensions  of a data: - 
• Remove  the redundant  dimensions  
• Only keep the most important dimensions  
Let us first try to understand  some  terms: - 
Variance : It is a measure of the variability or it simply measures how spread the data set is.  
Mathematically,  it is the average  squared  deviation  from  the mea

### **Splitting of document**

[Recursively split by character](https://python.langchain.com/docs/how_to/recursive_text_splitter/)

[Split by character](https://python.langchain.com/docs/how_to/character_text_splitter/)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# Split
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

In [ ]:
splits = text_splitter.split_documents(docs)

print(len(splits))
print(len(splits[0].page_content) )
splits[0].page_content

26
443


'1 \n \n \nN \n \n1 Principal Component Analysis \nIn real world data analysis tasks we analyze complex data i.e. multi dimensional data. We plot the  \ndata and find various patterns in it or use it to train some machine learning models.  One way to  \nthink about dimensions is that suppose you have an data point x , if we consider this data point as \na physical object then dimensions are merely a basis of view, like where is the data located when'

In [ ]:
splits[0]

Document(metadata={'source': '/content/pca_d1.pdf', 'page': 0}, page_content='1 \n \n \nN \n \n1 Principal Component Analysis \nIn real world data analysis tasks we analyze complex data i.e. multi dimensional data. We plot the  \ndata and find various patterns in it or use it to train some machine learning models.  One way to  \nthink about dimensions is that suppose you have an data point x , if we consider this data point as \na physical object then dimensions are merely a basis of view, like where is the data located when')

### **Embeddings**

Let's take our splits and embed them.

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cpu


In [ ]:
# Embedding Model

from langchain_huggingface import HuggingFaceEmbeddings

modelPath ="mixedbread-ai/mxbai-embed-large-v1"                  # Model card: https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1
                                                                 # Find other Emb. models at: https://huggingface.co/spaces/mteb/leaderboard

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device': device}      # cuda/cpu

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

embedding =  HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/114k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
embedding

HuggingFaceEmbeddings(model_name='mixedbread-ai/mxbai-embed-large-v1', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': False}, multi_process=False, show_progress=False)

### **Understanding similarity search with a toy example**

In [ ]:
sentence1 = "i like dogs"
sentence2 = "i like cats"
sentence3 = "the weather is ugly, too hot outside"

In [ ]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [ ]:
len(embedding1), len(embedding2), len(embedding3)

(1024, 1024, 1024)

In [ ]:
embedding1[:10]

[0.1741190254688263,
 0.15716786682605743,
 0.23070229589939117,
 -0.5954374670982361,
 -0.5540168881416321,
 -0.28742364048957825,
 0.25154373049736023,
 0.6992034316062927,
 0.43466925621032715,
 0.7160353064537048]

In [ ]:
import numpy as np

def cosine_similarity(vector1, vector2):
    # Ensure that the vectors are numpy arrays
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)

    # Calculate the dot product of the vectors
    dot_product = np.dot(vector1, vector2)

    # Calculate the magnitude (norm) of the vectors
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)

    # Compute cosine similarity
    if norm_vector1 == 0 or norm_vector2 == 0:
        return 0  # Avoid division by zero
    return dot_product / (norm_vector1 * norm_vector2)


In [ ]:
cosine_similarity(embedding1, embedding2), cosine_similarity(embedding1, embedding3), cosine_similarity(embedding2, embedding3)

(0.745220694977178, 0.3721567193745441, 0.35343557090974825)

### **Vectorstores**

In [ ]:
from langchain_chroma import Chroma       # Light-weight and in memory

In [ ]:
persist_directory = 'docs/chroma/'
!rm -rf ./docs/chroma  # remove old database files if any

In [ ]:
vectordb = Chroma.from_documents(
    documents=splits,                    # splits we created earlier
    embedding=embedding,
    persist_directory=persist_directory, # save the directory
)

In [ ]:
print(vectordb._collection.count()) # same as number of splits

26


### **Similarity Search in Vector store**

Algorithms for retrieving relevant chunks In Vector databases,

In vector databases, algorithms for retrieving relevant chunks to a query are often based on **similarity search techniques**, primarily using nearest neighbor search.

Here are some common approaches:

>**Approximate Nearest Neighbor (ANN) Search:** Vector databases frequently use ANN algorithms to improve efficiency when searching for vectors that
are close to the query vector.
>
>Popular **ANN** algorithms include:
>
>1. HNSW (Hierarchical Navigable Small World Graph): This is a graph-based approach that finds approximate nearest neighbors using a multi-
layered graph structure.
>
>2. Faiss: An open-source library developed by Facebook, which uses various algorithms for fast similarity search, such as Product Quantization and
Inverted File System (IVF).
>
>3. Annoy (Approximate Nearest Neighbors Oh Yeah): Developed by Spotify, it uses a forest of random projection trees for approximate nearest
neighbor search.


In [ ]:
question = "How does ensemble method works?"

In [ ]:
docs = vectordb.similarity_search(question, k=6)     # k --> No. of Document object to return

In [ ]:
print(len(docs))

for i in range(len(docs)):
    print(docs[i].page_content)
    print('='*140)

6
considered. The product is bought by the user when the combined ratings of the group is positive. 
The user gets a fairer idea about the product when all the ratings are combined. 
Here, the combination of ratings is done so that the decision making process of the user is made  
easy. 
Ensemble Methods refer to combining many different machine learning models in order to get a  
more powerful prediction. 
Thus, ensemble methods increase the accuracy of the predictions.
considered. The product is bought by the user when the combined ratings of the group is positive. 
The user gets a fairer idea about the product when all the ratings are combined. 
Here, the combination of ratings is done so that the decision making process of the user is made  
easy. 
Ensemble Methods refer to combining many different machine learning models in order to get a  
more powerful prediction. 
Thus, ensemble methods increase the accuracy of the predictions.
1  
 
Ensemble Methods 
Let us consider a real wor

### **Edge cases where failure may happen**

1. Lack of Diversity : Semantic search fetches all similar documents, but does not enforce diversity.

    - Notice that we're getting duplicate chunks (because of the duplicate `ens_d2.pdf` in the index). `docs[0]` and `docs[1]` are indentical.

  **Addressing Diversity - MMR (Maximum Marginal Relevance)**

Maximum Marginal Relevance (MMR) is a method used to retrieve relevant items to a query while avoiding redundancy. It does this by ensuring a balance between relevancy and diversity in the items retrieved.

<img src='https://miro.medium.com/v2/resize:fit:828/format:webp/1*U-9mPt5tBfPBPrwC4_oD1w.png'>

In [ ]:
question = 'How ensemble method works?'
docs = vectordb.similarity_search(question, k=3)     # Without MMR

print(len(docs))

for i in range(len(docs)):
    print(docs[i].page_content)
    print('='*140)

3
considered. The product is bought by the user when the combined ratings of the group is positive. 
The user gets a fairer idea about the product when all the ratings are combined. 
Here, the combination of ratings is done so that the decision making process of the user is made  
easy. 
Ensemble Methods refer to combining many different machine learning models in order to get a  
more powerful prediction. 
Thus, ensemble methods increase the accuracy of the predictions.
considered. The product is bought by the user when the combined ratings of the group is positive. 
The user gets a fairer idea about the product when all the ratings are combined. 
Here, the combination of ratings is done so that the decision making process of the user is made  
easy. 
Ensemble Methods refer to combining many different machine learning models in order to get a  
more powerful prediction. 
Thus, ensemble methods increase the accuracy of the predictions.
1  
 
Ensemble Methods 
Let us consider a real wor

**Example 1. Addressing Diversity - MMR-Maximum Marginal Relevance**

In [ ]:
docs_with_mmr = vectordb.max_marginal_relevance_search(question, k=3, fetch_k=6)   # With MMR

print(len(docs_with_mmr))

for i in range(len(docs_with_mmr)):
    print(docs_with_mmr[i].page_content)
    print('='*140)

3
considered. The product is bought by the user when the combined ratings of the group is positive. 
The user gets a fairer idea about the product when all the ratings are combined. 
Here, the combination of ratings is done so that the decision making process of the user is made  
easy. 
Ensemble Methods refer to combining many different machine learning models in order to get a  
more powerful prediction. 
Thus, ensemble methods increase the accuracy of the predictions.
1  
 
Ensemble Methods 
Let us consider a real world situation which uses Ensemble Methods, which is, when a user wants 
to buy a new product. Many users who have already purchased that product will have given either  
positive or negative ratings. If in the group, many users have given positive ratings, then the 
combined rating will be positive. Instead of a single rating, the ratings of the group of users is
Why use Ensemble Methods? 
Ensemble Methods are used in order to: 
• decrease variance (bagging) 
• decrease 

2. Lack of specificity:  The question may be from a particular doc but answer may contain information from other doc.

  **Addressing Specificity: Working with metadata - Manually**

  **Working with metadata using self-query retriever - Automatically**

**Example 2. Addressing Specificity: Working with metadata - Manually**

In [ ]:
# Without metadata information
question = "What is variance?"

docs = vectordb.similarity_search(question, k=5)

for doc in docs:
    print(doc.metadata)    # metadata contains information about from which doc the answer has been fetched

{'page': 0, 'source': '/content/pca_d1.pdf'}
{'page': 0, 'source': '/content/ens_d2.pdf'}
{'page': 0, 'source': '/content/ens_d2.pdf'}
{'page': 0, 'source': '/content/pca_d1.pdf'}
{'page': 1, 'source': '/content/pca_d1.pdf'}


We can filter the results based on metadata.

In [ ]:
# With metadata information
question = "what is the role of variance in pca?"
docs = vectordb.similarity_search(
    question,
    k=5,
    filter={"source":'/content/ens_d2.pdf'}     # manually passing metadata, using metadata filter.
)

for doc in docs:
    print(doc.metadata)

{'page': 1, 'source': '/content/ens_d2.pdf'}
{'page': 1, 'source': '/content/ens_d2.pdf'}
{'page': 0, 'source': '/content/ens_d2.pdf'}
{'page': 0, 'source': '/content/ens_d2.pdf'}
{'page': 0, 'source': '/content/ens_d2.pdf'}


In [ ]:
# With metadata information + MMR

docs_with_mmr = vectordb.max_marginal_relevance_search(question,
                                                       k=2,
                                                       fetch_k=5,
                                                       filter={"source":'/content/ens_d2.pdf'}     # manually passing metadata, using metadata filter.
                                                       )

In [ ]:
for i in range(len(docs_with_mmr)):
    print(docs_with_mmr[i].page_content)
    print('='*140)

subset of features is selected, further randomizing the tree. 
As a result, the bias of the forest increases slightly, but due to the averaging of less correlated  
trees, its variance decreases, resulting in an overall better model.
models. 
 
Variance 
Variance quantifies how the predictions made on same observation are different from each other. A  
high variance model will over -fit on your training population and perform badly on any observation  
beyond training. Thus, we aim at low variance.


[**Addressing Specificity -Automatically: Working with metadata using self-query retriever**](https://python.langchain.com/docs/how_to/self_query/)

### **Additional tricks: Compression**

Another approach for improving the quality of retrieved docs is compression. Information most relevant to a query may be buried in a document with a lot of irrelevant text. Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

[Contextual compression](https://python.langchain.com/docs/how_to/contextual_compression/) is meant to fix this.

## **Retrieval**

**[Vectorstore as a retriever](https://python.langchain.com/docs/how_to/vectorstore_retriever/)**

**Better Approach**

In [ ]:
# Without MMR
question = "What is principal component analysis?"
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
docs = retriever.invoke(question)
docs

[Document(metadata={'page': 1, 'source': '/content/pca_d1.pdf'}, page_content='2 \n \n \n \nSo, what does Principal Component Analysis (PCA) do? \nPCA finds a new set of dimensions (or a set of basis of views) such that all the dimensions are  \northogonal (and hence linearly independent) and ranked according to the variance of data along  \nthem. It means more important principle axis occurs first. (more important = more variance/more  \nspread out data) \n \nHow does PCA work? \n• Calculate the covariance matrix X of data points.'),
 Document(metadata={'page': 0, 'source': '/content/pca_d1.pdf'}, page_content='1 \n \n \nN \n \n1 Principal Component Analysis \nIn real world data analysis tasks we analyze complex data i.e. multi dimensional data. We plot the  \ndata and find various patterns in it or use it to train some machine learning models.  One way to  \nthink about dimensions is that suppose you have an data point x , if we consider this data point as \na physical object then di

In [ ]:
# With MMR
retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k":5})
docs = retriever.invoke(question)
docs

[Document(metadata={'page': 1, 'source': '/content/pca_d1.pdf'}, page_content='2 \n \n \n \nSo, what does Principal Component Analysis (PCA) do? \nPCA finds a new set of dimensions (or a set of basis of views) such that all the dimensions are  \northogonal (and hence linearly independent) and ranked according to the variance of data along  \nthem. It means more important principle axis occurs first. (more important = more variance/more  \nspread out data) \n \nHow does PCA work? \n• Calculate the covariance matrix X of data points.'),
 Document(metadata={'page': 0, 'source': '/content/pca_d1.pdf'}, page_content='1 \n \n \nN \n \n1 Principal Component Analysis \nIn real world data analysis tasks we analyze complex data i.e. multi dimensional data. We plot the  \ndata and find various patterns in it or use it to train some machine learning models.  One way to  \nthink about dimensions is that suppose you have an data point x , if we consider this data point as \na physical object then di

## **Augmentation**

In [ ]:
from langchain_core.prompts import PromptTemplate                                    # To format prompts
from langchain_core.output_parsers import StrOutputParser                            # to transform the output of an LLM into a more usable format
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough          # Required by LCEL (LangChain Expression Language)

In [ ]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""

QA_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)

## **Creating final RAG Chain**

> <img src='https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2F63f8a8482c9ec06a8d7d1041514f87c06dd108a9-3442x942.png&w=3840&q=75' width=1200px>

[[Image source](https://www.pinecone.io/learn/series/langchain/langchain-expression-language/)]

Above figure describes the LCEL flow using `RunnableParallel` and `RunnablePassthrough`.

A Runnable is a **unit of execution** in the LangChain framework. It represents a specific task or operation that can be performed.

Examples of Runnables include data transformations, computations, or any other operation that can be **expressed** in the LCEL(LangChain expression language).

[Runnable Lambdas](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html) is a LangChain abstraction that allows us to turn Python functions into **pipe-compatible functions**, similar to the Runnable class.

[RunnablePassthrough](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html) on its own allows you to pass inputs unchanged. This typically is **used in conjuction with [RunnableParallel](https://python.langchain.com/v0.1/docs/expression_language/interface/#parallelism)** to pass data through to a new key in the map.

The **RunnableParallel** object allows us to define multiple values and operations, and run them all in parallel.

The **RunnablePassthrough** object is used as a “passthrough” that takes any input to the current component ('retrieval' in above figure) and allows us to provide it in the component output via the “question” key or any other custom key.

In [ ]:
retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 7, "fetch_k":15})
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7ef6b13e5060>, search_type='mmr', search_kwargs={'k': 7, 'fetch_k': 15})

In [ ]:
retrieval = RunnableParallel(
    {
        "context": RunnablePassthrough(context= lambda x: x["question"] | retriever),
        "question": RunnablePassthrough()
        }
    )

In [ ]:
# RAG Chain

rag_chain = (retrieval                     # Retrieval
             | QA_PROMPT                   # Augmentation
             | llm                         # Generation
             | StrOutputParser()
             )

In [ ]:
response = rag_chain.invoke({"question": "What is PCA ?"})

response

' Principal Component Analysis (PCA) is a statistical technique used to reduce the dimensionality of a large dataset by transforming it into a new set of variables, called principal components, which are uncorrelated and explain the maximum amount of variance in the original data. It is commonly used in machine learning and data analysis to simplify complex datasets and extract meaningful insights.'

In [ ]:
response = rag_chain.invoke({"question": "What is principal component analysis?"})

response

' Principal component analysis (PCA) is a statistical technique used to reduce the dimensionality of a large dataset by transforming it into a new set of variables called principal components. These components are linear combinations of the original variables and are ordered by their variance, with the first component explaining the most variability in the data. PCA can help to identify patterns and structure in complex datasets, as well as to reduce the computational burden of analyzing high-dimensional data. It is commonly used in fields such as finance, marketing, and engineering.'

In [ ]:
response = rag_chain.invoke({"question": "How ensemble method works?"})

print(response)

 Ensemble methods are a type of machine learning technique that combines multiple models to make a more accurate prediction. The idea behind ensemble methods is that by combining the predictions of multiple models, we can reduce the variance and improve the overall accuracy of the model.

There are several types of ensemble methods, including:

1. Bagging (Bootstrap Aggregating): In bagging, multiple models are trained on different subsets of the training data. These models are then combined to make a final prediction.

2. Boosting: In boosting, multiple models are trained sequentially, with each model focusing on the errors made by the previous model. This helps to reduce the overall error rate.

3. Stacking: In stacking, multiple models are trained on different features or subsets of the data. These models are then combined using a meta-model to make a final prediction.

Ensemble methods can be particularly useful in situations where the data is complex or noisy, as they can help to 

In [ ]:
# For queries that is not in documents
response = rag_chain.invoke({"question": "Who is the CEO of OpenAI "})

print(response)           # It should return "I don't know. Thanks for asking!". The open-source model is not adhering to the prompt.

 Elon Musk co-founded OpenAI in 2015, but he stepped down from his role as CEO in 2019. The current CEO of OpenAI is Greg Brockman. Thanks for asking!


**To check what is being retreived from the retriever**

In [ ]:
chain_retriever = RunnablePassthrough() | retriever

In [ ]:
chain_retriever.invoke("What is principal component analysis?")

[Document(metadata={'page': 1, 'source': '/content/pca_d1.pdf'}, page_content='2 \n \n \n \nSo, what does Principal Component Analysis (PCA) do? \nPCA finds a new set of dimensions (or a set of basis of views) such that all the dimensions are  \northogonal (and hence linearly independent) and ranked according to the variance of data along  \nthem. It means more important principle axis occurs first. (more important = more variance/more  \nspread out data) \n \nHow does PCA work? \n• Calculate the covariance matrix X of data points.'),
 Document(metadata={'page': 0, 'source': '/content/pca_d1.pdf'}, page_content='1 \n \n \nN \n \n1 Principal Component Analysis \nIn real world data analysis tasks we analyze complex data i.e. multi dimensional data. We plot the  \ndata and find various patterns in it or use it to train some machine learning models.  One way to  \nthink about dimensions is that suppose you have an data point x , if we consider this data point as \na physical object then di

In [ ]:
chain_retriever.invoke("How ensemble method works?")

[Document(metadata={'page': 0, 'source': '/content/ens_d2.pdf'}, page_content='considered. The product is bought by the user when the combined ratings of the group is positive. \nThe user gets a fairer idea about the product when all the ratings are combined. \nHere, the combination of ratings is done so that the decision making process of the user is made  \neasy. \nEnsemble Methods refer to combining many different machine learning models in order to get a  \nmore powerful prediction. \nThus, ensemble methods increase the accuracy of the predictions.'),
 Document(metadata={'page': 0, 'source': '/content/ens_d2.pdf'}, page_content='1  \n \nEnsemble Methods \nLet us consider a real world situation which uses Ensemble Methods, which is, when a user wants \nto buy a new product. Many users who have already purchased that product will have given either  \npositive or negative ratings. If in the group, many users have given positive ratings, then the \ncombined rating will be positive. Ins

[**Details of Chroma through LangChain**](https://python.langchain.com/docs/integrations/vectorstores/chroma/)

## Reusing Vector DB

### **Download the vector DB**

In [ ]:
# Zip the entire folder
!zip -r /content/docs.zip /content/docs

In [ ]:
from google.colab import files
files.download("/content/docs.zip")

### **Upload the vector db from previous step and unzip**

In [ ]:
!unzip /content/docs.zip  -d /

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embedding =  HuggingFaceEmbeddings(
    model_name="mixedbread-ai/mxbai-embed-large-v1",                             # Provide the pre-trained model's path
    model_kwargs={'device': "cuda" if torch.cuda.is_available() else "cpu"},     # Pass the model configuration options
    encode_kwargs={'normalize_embeddings': False}                                # Pass the encoding options
)

vectordb = Chroma(persist_directory = 'docs/chroma/',
                  embedding_function = embedding
                  )